In [98]:
#Copy and collect the right pictures from the source:
import pandas as pd
import numpy as np

missing_anchors = False
missing_unlabeled = False
missing_unlabeled_conv = False

In [82]:
# Paths to full-collection of pictures
# (Jonathan's pc):
path_2017 = r"C:\Users\COBOD\3D Objects\ISIC-2017_Training_Data"
path_2020 = r"C:\Users\COBOD\3D Objects\ISIC_2020_Training_Dicom.zip"

In [83]:
# Path to destination of collections:
#C:\Users\COBOD\OneDrive\CBS doku\ITU\3semester\Project\Data
dest_2017 = "anchor_images"
dest_2020 = "unlabeled_images"

In [84]:
# Create list of anchor images needed:
df_anchor = pd.read_csv("ISIC-2017_anchor_images.csv")
images_2017 = df_anchor["image_id"].values.tolist()
#print(images_2017)

In [85]:
# Create list of unlabeled images needed:
df_unlabeled = pd.read_csv("ISIC_2020_unlabeled_images.csv")
images_2020 = df_unlabeled["image_id"].values.tolist()
#print(images_2020)

In [96]:
import shutil, zipfile
#archive = zipfile.ZipFile(path_2017c, 'r')
#imgdata = archive.read('ISIC_0000143.jpg')

# Example of how to define the file
#file = path_2017 + "\\" + "ISIC_0000143.jpg"

# Create anchor pictures:
# From folder (not Zip)
print(missing_anchors)
if (missing_anchors):
    print("Begin: copying and collecting anchor images ...")
    for i in images_2017:
        file = path_2017 + "\\" + i + ".jpg"
        # Save the raw image "un comment this"
        shutil.copy(file, dest_2017)
        
        # Rescale the image sizes to be 256 x 256 pixels
        dest_resize = r"C:\Users\COBOD\OneDrive\CBS doku\ITU\3semester\Project\GitHubs\Project\crowdsim\Extra\Data\anchor_images\\"
        basewidth = 256
        img = Image.open(file)
        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
        img = img.resize((basewidth,hsize), Image.ANTIALIAS)
        img.save(dest_resize + "resized\\" + i + "_resized.jpg", optimize=True, quality=85)
    print("Done: copying and collecting anchor images")


True
Begin: copying and collecting anchor images ...
Done: copying and collecting anchor images


In [97]:
# Create unlabeled pictures:
# From Zip-file

print(missing_unlabeled)
if(missing_unlabeled):
    archive = zipfile.ZipFile(path_2020, 'r')
    file = "train/" + images_2020[0] + ".dcm"
    #print(file)
    #archive.extract(file, dest_2020)

    print("Begin: copying and collecting unlabeled images ...")
    for i in images_2020:
        file = "train/" + i + ".dcm"
        archive.extract(file, dest_2020)
    print("Done: copying and collecting unlabeled images")
    print("You need to convert the dcm to jpg!")
    

True
Begin: copying and collecting unlabeled images ...
Done: copying and collecting unlabeled images
You need to convert the dcm to jpg!


In [99]:
# Resizing the unlabeled images and saved seperately
# USing https://pycad.co/how-to-convert-a-dicom-image-into-jpg-or-png/
import os
import glob
import pydicom
import imageio
from PIL import Image

## THANK YOU!!! https://gist.github.com/Quasimondo/c3590226c924a06b276d606f4f189639
def YUV2RGB( yuv ):
      
    m = np.array([[ 1.0, 1.0, 1.0],
                 [-0.000007154783816076815, -0.3441331386566162, 1.7720025777816772],
                 [ 1.4019975662231445, -0.7141380310058594 , 0.00001542569043522235] ])
    
    rgb = np.dot(yuv,m)
    rgb[:,:,0]-=179.45477266423404
    rgb[:,:,1]+=135.45870971679688
    rgb[:,:,2]-=226.8183044444304
    return rgb

def DCM2JPG(path_to_use):
    dic_path = path_to_use + "train\\"

    test_list = [os.path.basename(x) for x in glob.glob(dic_path + './*.dcm')]
    print("Start conversion ...")
    for f in test_list:
        file_path = dic_path +  f
        print("Converting and resizing file:", f)
        
        # Load the DICOM file
        ds = pydicom.dcmread(file_path, force=True)
        #print(ds)
        
        # Convert the YUV colors to RGB (human readable)
        ds_rgb = YUV2RGB(ds.pixel_array)
        
        # Scale date to uint8
        ds_rgb = ds_rgb / ds_rgb.max() #normalizes data in range 0 - 255
        ds_rgb = 255 * ds_rgb
        ds_rgb = ds_rgb.clip(min=0)
        ds_rgb = ds_rgb.astype(np.uint8)
        
        # Save the image as JPG (raw)
        file_dest = dic_path[:-7] + "converted\\" + f[:-4] + ".jpg"
        imageio.imwrite(file_dest, ds_rgb, format="jpg")
        
        # Rescale the image sizes to be 256 x 256 pixels
        basewidth = 256
        img = Image.open(file_dest)
        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
        img = img.resize((basewidth,hsize), Image.ANTIALIAS)
        img.save(dic_path[:-7] + "resized\\" + f[:-4] + "_resized.jpg", optimize=True, quality=85)
    print("Conversion finished!")

if missing_unlabeled_conv:
    path = r"C:\Users\COBOD\OneDrive\CBS doku\ITU\3semester\Project\GitHubs\Project\crowdsim\Extra\Data\unlabeled_images\\"
    DCM2JPG(path)



Start conversion ...
Converting and resizing file: ISIC_0149568.dcm
Converting and resizing file: ISIC_0162708.dcm
Converting and resizing file: ISIC_0177860.dcm
Converting and resizing file: ISIC_0207268.dcm
Converting and resizing file: ISIC_0214635.dcm
Converting and resizing file: ISIC_0224681.dcm
Converting and resizing file: ISIC_0236510.dcm
Converting and resizing file: ISIC_0250839.dcm
Converting and resizing file: ISIC_0272509.dcm
Converting and resizing file: ISIC_0274382.dcm
Converting and resizing file: ISIC_0333091.dcm
Converting and resizing file: ISIC_0351666.dcm
Converting and resizing file: ISIC_0361341.dcm
Converting and resizing file: ISIC_0489267.dcm
Converting and resizing file: ISIC_0502419.dcm
Converting and resizing file: ISIC_0504165.dcm
Converting and resizing file: ISIC_0516183.dcm
Converting and resizing file: ISIC_0533122.dcm
Converting and resizing file: ISIC_0559335.dcm
Converting and resizing file: ISIC_0572205.dcm
Converting and resizing file: ISIC_0583